# Phase 1: 分析用ダミーデータの生成
このノートブックでは、保険業界のNPS（顧客推奨度）分析に使用するサンプルデータを生成します。
単なるランダムデータではなく、実務的なインサイトが得られるよう「特定の属性に偏り」を持たせたロジックを組み込みます。

### データの主な特徴
- **自社（A社）**: 高齢層×対面営業に強み。若年層×ネットに課題。
- **競合（B社）**: ネットチャネルの満足度が高い。
- **契約年数**: 更新時期（3-5年）や長期契約（10年以上）によるスコア変動。

In [8]:
import pandas as pd
import numpy as np
import os

# 再現性のためのシード設定
np.random.seed(42)

# 保存先ディレクトリの作成
os.makedirs('../data', exist_ok=True)

# サンプル設定
N_SAMPLES = 2000

## 1. 基本属性の生成
顧客ID、契約会社、販売チャネル、年代、契約年数をランダムに生成します。

In [12]:
# 2. 基本属性の生成（出現確率に重み付けを追加）
customer_ids = [f"CID{i:04d}" for i in range(1, N_SAMPLES + 1)]
companies = np.random.choice(["A社(自社)", "B社", "C社"], N_SAMPLES, p=[0.4, 0.3, 0.3])

# チャネルと年代に相関を持たせたいが、まずは独立して生成し重みを調整
channels = np.random.choice(["営業職員", "ネット"], N_SAMPLES, p=[0.6, 0.4])
ages = np.random.choice(
    [20, 30, 40, 50, 60, 70, 80], 
    N_SAMPLES, 
    p=[0.05, 0.15, 0.20, 0.25, 0.20, 0.10, 0.05]
)
tenure_years = np.random.randint(1, 21, N_SAMPLES)

df = pd.DataFrame({
    "customer_id": customer_ids,
    "company": companies,
    "channel": channels,
    "age_group": ages,
    "tenure_year": tenure_years
})

## 2. ビジネスロジックに基づいたNPSスコアの生成
各属性の組み合わせに応じて、スコア（0-10）が算出されるロジックを適用します。

In [13]:
def calculate_nps(row):
    # ベーススコア
    score = np.random.normal(6, 1.5)
    
    # ロジック適用
    if row['company'] == "A社(自社)":
        if row['age_group'] >= 60 and row['channel'] == "営業職員":
            score += 2.0  # 強み
        if row['age_group'] <= 30 and row['channel'] == "ネット":
            score -= 1.5  # 弱点
    elif row['company'] == "B社" and row['channel'] == "ネット":
        score += 1.5      # 競合の強み

    # 契約年数による補正
    if 3 <= row['tenure_year'] <= 5:
        score -= 1.0  # 更新期の不満
    elif row['tenure_year'] >= 10:
        score += 0.5  # 長期継続の信頼

    return int(np.clip(round(score), 0, 10))

df['nps_score'] = df.apply(calculate_nps, axis=1)

# NPS区分の定義
df['nps_segment'] = pd.cut(df['nps_score'], 
                           bins=[-1, 6, 8, 10], 
                           labels=['批判者', '中立者', '推奨者'])

## 3. データの検証と保存
生成されたデータが意図した傾向（A社のチャネル別スコアなど）になっているか確認し、CSVとして出力します。

In [14]:
# 簡易集計で傾向確認
print(df.groupby(['company', 'channel'])['nps_score'].mean().unstack())

# 保存
df.to_csv("../data/dummy_nps_data.csv", index=False, encoding="utf-8-sig")
print("CSV successfully saved to ../data/dummy_nps_data.csv")

channel       ネット      営業職員
company                    
A社(自社)   5.885057  6.757050
B社       7.675000  6.191436
C社       5.887892  6.093656
CSV successfully saved to ../data/dummy_nps_data.csv
